In [ ]:
# Removes lint errors from VS Code
from typing import Dict, TYPE_CHECKING, Tuple, List

if TYPE_CHECKING:
    import kedro

    catalog: kedro.io.data_catalog.DataCatalog
    session: kedro.framework.session.session.KedroSession
    pipelines: Dict[str, kedro.pipeline.pipeline.Pipeline]


In [ ]:
import os

VIEW = os.getenv("DATASET_VIEW") or "tab_adult"
TABLE = os.getenv("DATASET_TABLE") or "table"
MULTI_PROCESS = (
    os.getenv("MULTI_PROCESS") if os.getenv("MULTI_PROCESS") is not None else True
)

import numpy as np
import pandas as pd
from importlib import reload

from pasteur.transform import TableTransformer

bhr: pd.DataFrame = catalog.load(f"{VIEW}.wrk.bhr_{TABLE}")
trn: TableTransformer = catalog.load(f"{VIEW}.wrk.trn_{TABLE}")
random_state = catalog.load("params:random_state")


In [ ]:
bhr.head()

,age_0,age_1,age_2,age_3,age_4,age_5,workclass,fnlwgt_0,fnlwgt_1,fnlwgt_2,...,capital-loss_3,capital-loss_4,capital-loss_5,hours-per-week_0,hours-per-week_1,hours-per-week_2,hours-per-week_3,hours-per-week_4,hours-per-week_5,native-country
id,,,,,,,,,,,,,,,,,,,,,
4,False,False,True,False,False,False,4,True,True,True,...,False,False,False,False,False,True,True,False,False,5
6,False,True,True,True,False,False,4,True,True,False,...,False,False,False,False,False,True,False,False,False,23
7,True,True,True,True,False,False,6,False,False,True,...,False,False,False,False,True,True,True,False,False,39
9,False,True,False,True,False,False,4,True,True,False,...,False,False,False,False,False,True,True,False,False,39
10,False,False,False,True,False,False,4,True,False,True,...,False,False,False,True,False,False,True,True,False,39


In [ ]:
attr_str = trn.get_attributes("bhr", bhr)
cols = list(bhr.columns)

attr = []
for a_cols in attr_str.values():
    attr.append([cols.index(col) for col in a_cols])

attr

[
    [5, 4, 3, 2, 1, 0],
    [12, 11, 10, 9, 8, 7],
    [19, 18, 17, 16, 15, 14],
    [30, 29, 28, 27, 26, 25],
    [36, 35, 34, 33, 32, 31],
    [42, 41, 40, 39, 38, 37],
    [6],
    [21],
    [20],
    [23],
    [22],
    [43],
    [24],
    [13]
]

In [ ]:
data = bhr.to_numpy(dtype="int16")
data.shape

(13024, 44)

In [ ]:
domain = data.max(axis=0) + 1
domain

array([ 2,  2,  2,  2,  2,  2,  9,  2,  2,  2,  2,  2,  2, 16,  2,  2,  2,
        2,  2,  2,  7, 15,  6,  5,  2,  2,  2,  2,  2,  1,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2, 42], dtype=int16)

In [ ]:
%load_ext line_profiler

In [ ]:
# import torch

# def calc_mutual_info(data: torch.Tensor, domain: np.ndarray, x: list[int], p: list[int]):
#     sub_data = data[:, x + p]
#     sub_domain = domain[x + p].tolist()
#     margin = torch.histogramdd(sub_data, sub_domain)
#     margin /= margin.sum()
#     margin += 1e-6
#     margin /= margin.sum()

#     x_idx = tuple(range(len(x)))
#     p_idx = tuple(range(-len(p), 0))

#     x_mar = np.sum(margin, axis=p_idx).reshape(-1)
#     p_mar = np.sum(margin, axis=x_idx).reshape(-1)
#     j_mar = margin.reshape((len(x_mar), len(p_mar)))

#     # margin.reshape((len(x_mar), len(p_mar)))
#     # margin.shape|
#     return np.sum(j_mar*np.log2(j_mar/np.outer(x_mar, p_mar)))

# x = [11, 12, 15]
# p = [22, 23, 24]
# # %lprun -f calc_mutual_info calc_mutual_info(torch.from_numpy(data), torch.from_numpy(domain), x, p)
# calc_mutual_info(torch.from_numpy(data), domain, x, p)

In [ ]:
import jax.numpy as jnp

def calc_mutual_info(data: jnp.ndarray, domain: jnp.ndarray, x: list[int], p: list[int]):
    sub_data = data[:, jnp.asarray(x + p)]
    sub_domain = domain[jnp.asarray(x + p)]
    margin, _ = jnp.histogramdd(sub_data, sub_domain)
    margin /= margin.sum()
    margin += 1e-6
    margin /= margin.sum()

    x_idx = tuple(range(len(x)))
    p_idx = tuple(range(-len(p), 0))

    x_mar = jnp.sum(margin, axis=p_idx).reshape(-1)
    p_mar = jnp.sum(margin, axis=x_idx).reshape(-1)
    j_mar = margin.reshape((len(x_mar), len(p_mar)))

    # margin.reshape((len(x_mar), len(p_mar)))
    # margin.shape|
    return jnp.sum(j_mar*jnp.log2(j_mar/jnp.outer(x_mar, p_mar)))

x = [11, 12, 15]
p = [22, 23, 24]
# %lprun -f calc_mutual_info calc_mutual_info(data, domain, x, p)
jdata = jnp.asarray(data)
jdomain = jnp.asarray(domain)
%timeit calc_mutual_info(jdata, jdomain, x, p)

2000-01-01 00:00:00,000715: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2000-01-01 00:00:00,000608: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2000-01-01 00:00:00,000301: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


[08/04/22 18:08:43] INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find ]8;id=571028;file:///mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/jax/_src/lib/xla_bridge.py\xla_bridge.py]8;;\:]8;id=327274;file:///mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/jax/_src/lib/xla_bridge.py#328\328]8;;\
                             driver in registry given worker:                                                      

[08/04/22 18:08:44] INFO     Unable to initialize backend 'rocm': NOT_FOUND: Could not find       ]8;id=814139;file:///mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/jax/_src/lib/xla_bridge.py\xla_bridge.py]8;;\:]8;id=337728;file:///mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/jax/_src/lib/xla_bridge.py#328\328]8;;\
                             registered platform with name: "rocm". Available platform names are:                  
                             CUDA Interpreter Host                                                                 

                    INFO     Unable to initialize backend 'tpu': module 'jaxlib.xla_extension'    ]8;id=136575;file:///mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/jax/_src/lib/xla_bridge.py\xla_bridge.py]8;;\:]8;id=255816;file:///mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/jax/_src/lib/xla_bridge.py#328\328]8;;\
                             has no attribute 'get_tpu_client'                                                     

2000-01-01 00:00:00,000659: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2000-01-01 00:00:00,000161: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2000-01-01 00:00:00,000955: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to 

91.3 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def calc_mutual_info(data: np.ndarray, domain: np.ndarray, x: list[int], p: list[int]):
    sub_data = data[:, x + p]
    sub_domain = domain[x + p]
    margin, _ = np.histogramdd(sub_data, sub_domain)
    margin /= margin.sum()
    margin += 1e-24

    x_idx = tuple(range(len(x)))
    p_idx = tuple(range(-len(p), 0))

    x_mar = np.sum(margin, axis=p_idx).reshape(-1)
    p_mar = np.sum(margin, axis=x_idx).reshape(-1)
    j_mar = margin.reshape((len(x_mar), len(p_mar)))

    # margin.reshape((len(x_mar), len(p_mar)))
    # margin.shape|
    return np.sum(j_mar*np.log(j_mar/np.outer(x_mar, p_mar)))

x = [11, 12, 13, 14]
p = [22, 23, 24, 25]
# %lprun -f calc_mutual_info calc_mutual_info(data, domain, x, p)
dt = data.astype(dtype="int16")
%timeit calc_mutual_info(dt, domain, x, p)
# calc_mutual_info(data, domain, x, p)

4.65 ms ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
from sklearn.metrics import mutual_info_score

In [ ]:
%%timeit
true = data[:, 11] * domain[12] + data[:, 12]
pred = data[:, 22] * domain[23] + data[:, 23]

# true = data[:, x]
mutual_info_score(pred,true)

1.73 ms ± 7.82 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
